<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Word2vect


In [1]:
import numpy as np

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [4]:
a = corpus[0].split(" ")
b = corpus[1].split(" ")
c = corpus[2].split(" ")
unicos = np.sort(np.unique(np.array(a + b + c)))
unicos

array(['de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas',
       'que'], dtype='<U7')

### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [5]:
indices = np.zeros( (3, len(unicos)) )

In [6]:
indices_a = np.intersect1d(unicos, a, return_indices=True)[1]
indices_b = np.intersect1d(unicos, b, return_indices=True)[1]
indices_c = np.intersect1d(unicos, c, return_indices=True)[1]

In [7]:
indices[0,indices_a] = 1
indices[1, indices_b] = 1
indices[2, indices_c] = 1

In [8]:
indices

array([[0., 1., 0., 1., 0., 1., 0., 0., 1.],
       [1., 1., 1., 1., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 1., 0.]])

### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [33]:
frequency = np.copy(indices)

In [34]:
a0 = np.unique(a, return_index=True, return_counts=True)[0]
a2 = np.unique(a, return_index=True, return_counts=True)[2]
repetidas_a = a0[np.argwhere(a2>1)]
indice_repetidas_a = np.intersect1d(unicos, a0[np.argwhere(a2>1)], return_indices=True)[1]

In [35]:
b0 = np.unique(b, return_index=True, return_counts=True)[0]
b2 = np.unique(b, return_index=True, return_counts=True)[2]
repetidas_b = b0[np.argwhere(b2>1)]
indice_repetidas_b = np.intersect1d(unicos, b0[np.argwhere(b2>1)], return_indices=True)[1]

In [36]:
c0 = np.unique(c, return_index=True, return_counts=True)[0]
c2 = np.unique(c, return_index=True, return_counts=True)[2]
repetidas_c = c0[np.argwhere(c2>1)]
indice_repetidas_c = np.intersect1d(unicos, c0[np.argwhere(c2>1)], return_indices=True)[1]

In [37]:
frequency[0, indice_repetidas_a] = frequency[0, indice_repetidas_a] * a2[np.argwhere(a2>1)]
frequency[1, indice_repetidas_b] = frequency[1, indice_repetidas_b] * b2[np.argwhere(b2>1)]
frequency[2, indice_repetidas_c] = frequency[2, indice_repetidas_c] * c2[np.argwhere(c2>1)]

In [38]:
frequency

array([[0., 1., 0., 1., 0., 1., 0., 0., 1.],
       [1., 1., 1., 1., 0., 1., 2., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 1., 0.]])

### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno